# Evn

In [ ]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [ ]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)

# One-hot

In [ ]:
# one hot encoding text
text = """나는 책을 샀다
나는 책을 본다
나는 책을 팔았다
나는 책을 서점에서 샀다
나는 책을 도서관에서 본다"""

In [ ]:
tokens = text.split()
tokens

In [ ]:
words = list(dict.fromkeys(tokens))
words

In [ ]:
# 각 단어에 고유한 번호 부여한 dictionary 생성
word_to_id = {'[PAD]': 0, '[UNK]': 1}  # [PAD]: 길이 맞추는 용도, [UNK]: 알 수 없는 token
for word in words:
    word_to_id[word] = len(word_to_id)
word_to_id

In [ ]:
# 고유한 번호로 부터 단어를 찾을 수 있는 dictionary 생성
id_to_word = {word:_id for _id, word in word_to_id.items()}
id_to_word

In [ ]:
# 줄바꿈 단위로 문장 분리
sentences = text.split("\n")
sentences

In [ ]:
# 띄어쓰기 단위로 단어 분리
tokens = []
for sentence in sentences:
    # print(sentence)
    tokens.append(sentence.split())
tokens

In [ ]:
# tokens을 vocabulary의 고유 번호로 변경
token_ids = []
for line_token in tokens:
    token_ids.append([word_to_id[token] for token in line_token])
token_ids

In [ ]:
# one hot encoding
one_hot_encodings = []
for line_token in token_ids:
    print(line_token)
    one_hot_line = []  # 한 줄을 표현하는 벡터
    for id in line_token:
        ################################
        one_hot = [0] * len(word_to_id)  # 모두 0인 벡터를 만듬
        one_hot[id] = 1
        print(id, one_hot)
        one_hot_line.append(one_hot)
        ################################
    one_hot_encodings.append((one_hot_line))  # 라인을 전체 문서에 추가

In [ ]:
one_hot_encodings

In [ ]:
np.argmax(np.array(one_hot_encodings[4]), axis=-1)

In [ ]:
# tensorflow one hot
# token_ids가 앞 3개는 길이가 3 이지만 이후는 4로 tensorflow에서는 오류 발생 함
# depth는 vocabulary 크기
#############################
tf.one_hot(indices=token_ids, depth=len(word_to_id))
#############################

In [ ]:
token_ids[:3]

In [ ]:
# token_ids가 앞 3개개만 one_hot으로 변경
#############################
tf.one_hot(indices=token_ids[:3], depth=len(word_to_id))
#############################

In [ ]:
# 모두 길이가 4가 되도록 pad(0) 추가
pad_ids = []
for line in token_ids:
    line = line[:4]
    line += [0] * (4 - len(line))
    pad_ids.append(line)
pad_ids

In [ ]:
#############################
tf_one_hot_encodings = tf.one_hot(indices=pad_ids, depth=len(word_to_id))
tf_one_hot_encodings
#############################

# Embedding

In [ ]:
# 랜덤 매트릭스 생성
weights = np.random.randint(10, 100, size=(len(word_to_id), 4)) / 100
weights

In [ ]:
# 첫번째 문장 만 numpy array로 변경
one_hot_encoding_0 = np.array(one_hot_encodings[0])
one_hot_encoding_0

In [ ]:
print(token_ids[0])
print(weights[token_ids[0][0]])
print(weights[token_ids[0][1]])
print(weights[token_ids[0][2]])

In [ ]:
one_hot_encoding_0

In [ ]:
# one hot encoding은 matrix의 특정 row를 선택하는 것과 같은 결과
##################
np.matmul(one_hot_encoding_0, weights)
##################

In [ ]:
# tensorflow 에서도 tf.keras.layers.Embedding에서도 가능 함
# weights 초기화를 위해서 [matrix] 형태로 변환함
# embdding을 사용하면 token 번호를 바로 사용가능 함 현재 표준화된 방법
embedding = tf.keras.layers.Embedding(len(word_to_id), 4, weights=[weights])

In [ ]:
token_ids[0]

In [ ]:
embedding(np.array(token_ids[0]))

In [ ]:
##################
tf.gather(weights, np.array(token_ids[0]))
##################